In [1]:
import pandas as pd
import numpy as np

In [2]:
xls_IT_1 = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/preview_all.xlsx')
df_IT_1 = pd.read_excel(xls_IT_1)

In [3]:
#Benchmark trading data
xls_index = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Index_2011_2021.xlsx')
df_index = pd.read_excel(xls_index)
df_index.rename(columns={"TradingDate":"RegisterDate","IndexCode":"Exchange"}, inplace=True)

df_index['Exchange'] = df_index['Exchange'].replace(['VNINDEX','HNXINDEX','UPCOMINDEX'],['VNI','HNX','UPCOM'])

df_VNI = df_index[df_index['Exchange']=='VNI'].copy()
df_HNX = df_index[df_index['Exchange']=='HNX'].copy()
df_UPCOM = df_index[df_index['Exchange']=='UPCOM'].copy()

#Calculate daily ret of benchmarks
df_VNI['Rm_ret'] = df_VNI['CloseIndex'].pct_change(-1)
df_HNX['Rm_ret'] = df_HNX['CloseIndex'].pct_change(-1)
df_UPCOM['Rm_ret'] = df_UPCOM['CloseIndex'].pct_change(-1)

In [4]:
#Match Benchmark Ret HNX
df_IT_HNX = df_IT_1[df_IT_1['Exchange']=='HNX'].copy()

for x in range(-250,0):
    df_IT_HNX['Rm_T1_' + str(x)] = df_IT_HNX.set_index('RegisterDate').index.map(df_HNX.set_index('RegisterDate')['Rm_ret'].shift(x))

df_IT_HNX['Rm_T1_0'] = df_IT_HNX.set_index('RegisterDate').index.map(df_HNX.set_index('RegisterDate')['Rm_ret'])

for x in range(1,10):
    df_IT_HNX['Rm_T1_' + str(x)] = df_IT_HNX.set_index('RegisterDate').index.map(df_HNX.set_index('RegisterDate')['Rm_ret'].shift(x))

In [5]:
import statsmodels.api as sm

#Regress Ri on Rm

def regress(data):
    result_all=[]
    for i in range(len(data)):
        Y = list(data.iloc[i][32:253])
        X = list(data.iloc[i][292:513])
        X = sm.add_constant(X)
        result = sm.OLS(Y,X).fit()
        result_all.append(result.params)
    return result_all  

#Why slice [32:253] and [292:513]
#df_IT_HNX.columns.get_indexer(['Ri_T1_-250', 'Ri_T1_-30','Rm_T1_-250', 'Rm_T1_-30'])
#array([ 32, 252, 292, 512])

#Run for HNX
res_HNX = regress(df_IT_HNX)

In [6]:
#Insert alphas & betas into df_IT_VNI
df_IT_HNX['alpha'] = np.array(res_HNX)[:,0]
df_IT_HNX['beta'] = np.array(res_HNX)[:,1]

In [7]:
#Calculate sd of abnormal return of each stock in each event in period (-250,-30)

def sd_ar(data):
    sd_ar_all=[]
    for i in range(len(data)):
        R_i = list(data.iloc[i][32:253])
        R_m = list(data.iloc[i][292:513])
        ar = [s - data.iloc[i]['alpha'] - m*data.iloc[i]['beta'] for s,m in zip(R_i,R_m)]
        sd_ar_all.append([np.nanstd(ar)])
    return sd_ar_all

sd_HNX = sd_ar(df_IT_HNX)

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [8]:
#Insert sd of abnormal return into df_IT_VNI
df_IT_HNX['sd_ar'] = np.array(sd_HNX)[:,0]

In [9]:
df_IT_HNX = df_IT_HNX.drop(df_IT_HNX[df_IT_HNX.sd_ar >= 1].index)
df_IT_HNX = df_IT_HNX.drop(df_IT_HNX[df_IT_HNX.sd_ar.isnull()].index)

In [10]:
#Calculate cumulative abnormal return (-30,-1)

def cal_car_before(data):
    car_all=[]
    for i in range(len(data)):
        r_i = list(data.iloc[i][252:282])
        r_m = list(data.iloc[i][512:542])
        car = np.nansum([s - data.iloc[i]['alpha'] - m*data.iloc[i]['beta'] for s,m in zip(r_i,r_m)])
        car_all.append(car)
    return car_all

#Why slice [252:282] and [512:542]
#df_IT_HNX.columns.get_indexer(['Ri_T1_-30', 'Ri_T1_-1','Rm_T1_-30', 'Rm_T1_-1'])
#array([252, 281, 512, 541])

cal_before_HNX = cal_car_before(df_IT_HNX)

In [11]:
#Insert car before into df_IT_HNX
df_IT_HNX['car_before'] = np.array(cal_before_HNX)

In [12]:
df_IT_HNX['ar_T0'] = df_IT_HNX['Ri_T1_0'] - (df_IT_HNX['Rm_T1_0']*df_IT_HNX['beta']+df_IT_HNX['alpha'])

def cal_car_after(data):
    car_all=[]
    for i in range(len(data)):
        r_i = list(data.iloc[i][283:292])
        r_m = list(data.iloc[i][543:552])
        car = np.nansum([s - data.iloc[i]['alpha'] - m*data.iloc[i]['beta'] for s,m in zip(r_i,r_m)])
        car_all.append(car)
    return car_all

#Why slice [283:292] and [543:552]
#df_IT_HNX.columns.get_indexer(['Ri_T1_1', 'Ri_T1_9','Rm_T1_1', 'Rm_T1_9'])
#array([283, 291, 543, 551])

cal_after = cal_car_after(df_IT_HNX)

In [13]:
#Insert car after into df_IT_VNI
df_IT_HNX['car_after'] = np.array(cal_after)

df_IT_HNX['car_t_t+n'] = df_IT_HNX['car_after'] + df_IT_HNX['ar_T0']

In [14]:
#Calculate t-statistics for event study of each event, yet I don't use this result
df_IT_HNX['t-stat_before'] = df_IT_HNX['car_before']/(df_IT_HNX['sd_ar']*(30**0.5))
df_IT_HNX['t-stat_eventdate'] = df_IT_HNX['ar_T0']/df_IT_HNX['sd_ar']
df_IT_HNX['t-stat_after'] = df_IT_HNX['car_after']/(df_IT_HNX['sd_ar']*((df_IT_HNX['Diff_days']-1)**0.5))
df_IT_HNX['t-stat_t_t+n'] = df_IT_HNX['car_t_t+n']/(df_IT_HNX['sd_ar']*(df_IT_HNX['Diff_days']**0.5))

df_IT_HNX

,StockCode,RegisterDate,ResultReportDate,DateStartExpected,DateEndExpected,DateActionFrom,DateActionTo,TypeName,RegisterVolumeBefore,RegisterVolumeBeforePercent,...,beta,sd_ar,car_before,ar_T0,car_after,car_t_t+n,t-stat_before,t-stat_eventdate,t-stat_after,t-stat_t_t+n
27,B82,2012-02-01,2012-03-07,2012-02-06,2012-03-05,2012-02-06,2012-03-05,GD CD noi bo,10097,0.003366,...,0.411131,0.042165,-0.350060,0.067019,0.040266,0.107285,-1.515745,1.589443,0.675248,1.469003
33,BVS,2012-01-13,2012-03-13,2012-01-16,2012-03-08,2012-01-17,2012-03-06,GD CD noi bo,9500,0.000132,...,1.768239,0.021498,-0.311726,0.042249,0.000000,0.042249,-2.647413,1.965274,NaN,1.965274
34,CJC,2012-01-06,2012-03-13,2012-01-09,2012-03-08,2012-01-09,2012-03-06,GD CD noi bo,53250,0.026625,...,0.146802,0.027993,-0.010111,0.021857,0.000000,0.021857,-0.065944,0.780807,NaN,0.780807
96,BVG,2012-01-20,2012-05-02,2012-01-30,2012-03-29,2012-01-30,2012-03-30,GD CD noi bo,2225000,0.228183,...,1.096484,0.034721,-0.088056,0.038797,0.022708,0.061505,-0.463021,1.117391,0.292477,0.723166
103,BBS,2012-02-23,2012-05-08,2012-03-01,2012-04-27,2012-03-01,2012-04-27,GD CD noi bo,105429,0.026357,...,0.289828,0.041604,0.273364,-0.002852,-0.049464,-0.052315,1.199641,-0.068544,-0.594463,-0.562358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11869,TVC,2020-11-13,2020-12-04,2020-11-18,2020-12-17,2020-11-18,2020-11-27,GD cua nguoi lien quan,15604073,0.355788,...,0.419397,0.036683,0.197355,0.011345,0.027484,0.038829,0.982241,0.309279,0.529776,0.611123
11872,PSC,2020-11-04,2020-12-07,2020-11-09,2020-12-07,2020-11-09,2020-12-02,GD cua nguoi lien quan,85700,0.011903,...,-0.053290,0.033942,0.037086,-0.000151,-0.001685,-0.001836,0.199490,-0.004434,-0.035112,-0.031229
11889,NSH,2020-12-11,2020-12-16,2020-12-14,2021-01-12,2020-12-10,2020-12-10,GD cua nguoi lien quan,57204,0.002764,...,1.051196,0.049692,0.753278,-0.042381,0.000000,-0.042381,2.767646,-0.852884,NaN,-0.852884
11907,TVC,2020-12-11,2020-12-23,2020-12-14,2021-01-08,2020-12-14,2020-12-21,GD cua nguoi lien quan,16104073,0.349739,...,0.481178,0.037466,0.228084,0.017226,0.000000,0.017226,1.111463,0.459765,NaN,0.459765


In [15]:
df_IT_HNX['StockCode'].nunique()

384

In [16]:
df_HNX_buy = pd.DataFrame(df_IT_HNX[(df_IT_HNX['RegisterSellVolume'].isnull())])
df_HNX_sell = pd.DataFrame(df_IT_HNX[(df_IT_HNX['RegisterBuyVolume'].isnull())])

In [17]:
#Form groups of events for t-test & Wilcoxon test

import scipy
from scipy.stats import norm
from scipy.stats import wilcoxon

df_buy_1 = df_HNX_buy[(df_HNX_buy['Diff_days']==1)]
df_sell_1 = df_HNX_sell[(df_HNX_sell['Diff_days']==1)]

df_buy_2 = df_HNX_buy[(df_HNX_buy['Diff_days']==2)]
df_sell_2 = df_HNX_sell[(df_HNX_sell['Diff_days']==2)]

df_buy_3 = df_HNX_buy[(df_HNX_buy['Diff_days']==3)]
df_sell_3 = df_HNX_sell[(df_HNX_sell['Diff_days']==3)]

In [18]:
import scipy.integrate 

from scipy import stats

tstat_HNX_1_buy, pval_HNX_1_buy = scipy.stats.ttest_1samp(df_buy_1['car_t_t+n'],0,alternative='two-sided')
tstat_HNX_1_buy, pval_HNX_1_buy, df_buy_1['car_t_t+n'].mean(), df_buy_1['car_t_t+n'].std(), len(df_buy_1)

(1.382425818538836,
 0.16874387902685192,
 0.004378040864333187,
 0.040432610015725914,
 163)

In [19]:
tstat_HNX_1_sell, pval_HNX_1_sell = scipy.stats.ttest_1samp(df_sell_1['car_t_t+n'],0,alternative='two-sided')
tstat_HNX_1_sell, pval_HNX_1_sell, df_sell_1['car_t_t+n'].mean(), df_sell_1['car_t_t+n'].std(), len(df_sell_1)

(-0.1335807079348853,
 0.8938817295369765,
 -0.0003638879925858776,
 0.03685102136357,
 183)

In [20]:
tstat_HNX_2_buy, pval_HNX_2_buy = scipy.stats.ttest_1samp(df_buy_2['car_t_t+n'],0,alternative='two-sided')
tstat_HNX_2_buy, pval_HNX_2_buy, df_buy_2['car_t_t+n'].mean(), df_buy_2['car_t_t+n'].std(), len(df_buy_2)

(3.8071109561337075,
 0.00016063879891897303,
 0.009991129478517527,
 0.05498589685046139,
 439)

In [21]:
tstat_HNX_2_sell, pval_HNX_2_sell = scipy.stats.ttest_1samp(df_sell_2['car_t_t+n'],0,alternative='two-sided')
tstat_HNX_2_sell, pval_HNX_2_sell, df_sell_2['car_t_t+n'].mean(), df_sell_2['car_t_t+n'].std(), len(df_sell_2)

(-1.1009390508470167,
 0.2713944922152501,
 -0.0025940380337907275,
 0.05590713797450507,
 563)

In [22]:
tstat_HNX_3_buy, pval_HNX_3_buy = scipy.stats.ttest_1samp(df_buy_3['car_t_t+n'],0,alternative='two-sided')
tstat_HNX_3_buy, pval_HNX_3_buy, df_buy_3['car_t_t+n'].mean(), df_buy_3['car_t_t+n'].std(), len(df_buy_3)

(2.6195520036429776,
 0.009274420096127398,
 0.011650622850251025,
 0.07534649726745576,
 287)

In [23]:
tstat_HNX_3_sell, pval_HNX_3_sell = scipy.stats.ttest_1samp(df_sell_3['car_t_t+n'],0,alternative='two-sided')
tstat_HNX_3_sell, pval_HNX_3_sell, df_sell_3['car_t_t+n'].mean(), df_sell_3['car_t_t+n'].std(), len(df_sell_3)

(-0.6577117427152372,
 0.5111143843365493,
 -0.00249597271610211,
 0.07475153250744286,
 388)

In [24]:
import scipy
from scipy.stats import norm
from scipy.stats import wilcoxon

#The wilcoxon function below does not return z-score 
#so I have to reverse-engineer z-score from p-value: norm.ppf(p-value/2) as we conduct a two-sided test

w1b, p1b = wilcoxon(df_buy_1["car_t_t+n"])
w1b, p1b, norm.ppf(p1b/2), df_buy_1["car_t_t+n"].median()

(6313.0, 0.5398206638222165, -0.6130842045560476, -0.00023720057190935756)

In [25]:
w1s, p1s = wilcoxon(df_sell_1["car_t_t+n"])
w1s, p1s, norm.ppf(p1s/2), df_sell_1["car_t_t+n"].median()

(7725.0, 0.334159192868707, -0.9657701806927574, -0.0013258846571858264)

In [26]:
w2b, p2b = wilcoxon(df_buy_2["car_t_t+n"])
w2b, p2b, norm.ppf(p2b/2), df_buy_2["car_t_t+n"].median()

(39599.0, 0.0010847972356155084, -3.267557772302731, 0.0011935867436732286)

In [27]:
w2s, p2s = wilcoxon(df_sell_2["car_t_t+n"])
w2s, p2s, norm.ppf(p2s/2), df_sell_2["car_t_t+n"].median()

(69689.0, 0.012057405403036323, -2.5104597873282417, -0.0037167416242984636)

In [28]:
w3b, p3b = wilcoxon(df_buy_3["car_t_t+n"])
w3b, p3b, norm.ppf(p3b/2), df_buy_3["car_t_t+n"].median()

(17825.0, 0.04364850224825746, -2.017450687133552, 0.0012145461069267458)

In [29]:
w3s, p3s = wilcoxon(df_sell_3["car_t_t+n"])
w3s, p3s, norm.ppf(p3s/2), df_sell_3["car_t_t+n"].median()

(32590.0, 0.019986604617675607, -2.3265992478223545, -0.004687542248455979)